In [8]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler

In [67]:
df = pd.read_csv("../../feature1.csv")
# df.columns

Index(['Dati', 'Time', 'DEPTH', 'ENGINE_1_FLOWTEMPA',
       'ENGINE_1_FUEL_CONSUMPTION', 'ENGINE_2_FLOWTEMPA',
       'ENGINE_2_FUEL_CONSUMPTION', 'HEADING', 'LATITUDE', 'LONGITUDE',
       'PITCH_1', 'PITCH_2', 'POWER_1', 'POWER_2', 'SOG', 'SOG_SPEEDLOG_LONG',
       'SOG_SPEEDLOG_TRANS', 'SPEED_1', 'SPEED_2', 'STW', 'THRUST_1',
       'THRUST_2', 'TORQUE_1', 'TORQUE_2', 'WIND_ANGLE', 'WIND_SPEED',
       'WIND_ANGLE_TRUE', 'WIND_SPEED_TRUE', 'trip_id', 'MODE', 'datetime',
       'season', 'weekday', 'current', 'direction', 'pressure', 'rain',
       'snowfall', 'weathercode', 'is_weekday', 'wind_force', 'wind_direc',
       'effective_wind_factor', 'effective_wind', 'resist_ratio1',
       'resist_ratio2', 'adversarial'],
      dtype='object')

In [68]:
df["direction"] = df["direction"].apply(lambda x: 1 if x=="N-H" else 0)
df["MODE"] = df["MODE"].apply(lambda x: x-1)
# df = df[df["adversarial"]==0].drop("adversarial", axis=1)
# df = df[df["direction"]==1].drop("direction", axis=1)
df["Time2"] = df.groupby("trip_id")["Time"].rank(method="first", ascending=True)
df["is_weekday"] = df["is_weekday"].apply(lambda x: 1 if x==True else 0)

df["change_x_factor"] = np.cos((df.HEADING+90) * np.pi / 180)
df["change_y_factor"] = np.sin((df.HEADING-90) * np.pi / 180)
df["goal_long"] = df.direction.apply(lambda x: 0.9965111208024382 if x==1 else 0.0023259194650222526)
df["goal_lat"] = df.direction.apply(lambda x: 0.7729570345408661 if x==1 else 0)
df["prev_HEADING"] = df.HEADING.shift(periods=1)
df["turn"] = df.HEADING - df.prev_HEADING

In [69]:
df["POWER"] = (df["POWER_1"]+df["POWER_2"])/2
df["SPEED"] = (df["SPEED_1"]+df["SPEED_2"])/2
df["THRUST"] = (df["THRUST_1"]+df["THRUST_2"])/2
df["TORQUE"] = (df["TORQUE_1"]+df["TORQUE_2"])/2
df["PITCH"] = (df["PITCH_1"]+df["PITCH_2"])/2
df["resist_ratio"] = (df["resist_ratio1"]+df["resist_ratio2"])/2
df["FLOWTEMPA"] = (df["ENGINE_1_FLOWTEMPA"]+df["ENGINE_2_FLOWTEMPA"])/2
df["FC"] = (df["ENGINE_1_FUEL_CONSUMPTION"]+df["ENGINE_2_FUEL_CONSUMPTION"])/2

df = df.drop(['PITCH_1', 'PITCH_2', 'POWER_1', 'POWER_2', 'SOG_SPEEDLOG_LONG',
       'SOG_SPEEDLOG_TRANS',  'THRUST_1',
       'THRUST_2', 'TORQUE_1', 'TORQUE_2', 'datetime',
       'resist_ratio1', 'resist_ratio2', 'SPEED_1', 'SPEED_2',
       'ENGINE_1_FLOWTEMPA', 'ENGINE_2_FLOWTEMPA',
       "ENGINE_1_FUEL_CONSUMPTION", "ENGINE_2_FUEL_CONSUMPTION"], axis=1)


In [ ]:
df["dt"] = pd.to_datetime(df.Dati, format='%y%m%d_%H%M%S')
df["hour"] = df.dt.apply(lambda x: x.hour/24)

In [70]:
# get one_hot for column
def one_hot(df, cols, normalize = True):
    for col in cols:
        dummy = pd.get_dummies(df[col],prefix=col, drop_first=True)
        columns = dummy.columns
        dummy[col] = 0
        i = 1
        for x in columns:
            # dummy["tmp"] = dummy[col]
            dummy[col] = dummy[col] + dummy[x]*i
            i = i*2
        max = dummy[col].max()
        if normalize:
            df[col] = dummy[col] / max
        else:
            df[col] = dummy[col]
    return df

In [71]:
minmax_scaler = MinMaxScaler((0,1))

transform_cols = [ 'current', 'rain', 'snowfall', "pressure", 'wind_force', "resist_ratio",
       'FC', "LATITUDE", 'LONGITUDE', 'SOG', "DEPTH", "SPEED"]
minmax_scaler = minmax_scaler.fit(df[transform_cols].values)
import pickle
pickle.dump(minmax_scaler, open('minmax_scaler.pkl', 'wb'))
df[transform_cols] = minmax_scaler.transform(df[transform_cols])
# df[transform_cols] = scaler2.fit_transform(df[transform_cols])
# df[["FC", "SOG", "SPEED"]] = df[["FC", "SOG", "SPEED"]].apply(lambda x: x**2)
# transform_cols
df[["FC2"]] = df[["FC"]].apply(lambda x: x**2)
df[["SOG4"]] = df[["SOG"]].apply(lambda x: x**4)

df["prev_SOG"] = df.SOG.shift(periods=1)
df["acceleration"] = ((df.SOG - df.prev_SOG))
df["distance"] = ((df["goal_long"]-df["LONGITUDE"])**2 + \
                             (df["goal_lat"]-df["LATITUDE"])**2 )**0.5

df["disp_x"] = df["change_x_factor"] * df["SOG"]
df["disp_y"] = df["change_y_factor"] * df["SOG"]

df = one_hot(df, ["season", "weathercode", "wind_direc"], normalize = True)
df = df[df["adversarial"]==0]

In [73]:
# H-N
d1 = df[df.direction==1][["LONGITUDE", "LATITUDE","FC", "trip_id"]] # h-n
d2 = df[df.direction==0][["LONGITUDE", "LATITUDE","FC", "trip_id"]] # n-h

# get the avg location of the top 1 percent trip
def get_top1_location(d1):
    total = d1.trip_id.nunique() // 100
    top_1_trips = list(d1.groupby("trip_id").FC.sum().sort_values()[:total].index)
    i = top_1_trips.pop(0)
    hn_top = d1[d1.trip_id==i].reset_index(drop=True)
    if (len(hn_top) < 200):
        for j in range(200-len(hn_top)):
            hn_top = hn_top.append(hn_top.tail(1)).reset_index(drop=True)
    for i in top_1_trips:
        tmp_df2 = df[df.trip_id==i].reset_index(drop=True)
        if (len(tmp_df2) < 200):
            for j in range(200-len(tmp_df2)):
                tmp_df2 = tmp_df2.append(tmp_df2.tail(1)).reset_index(drop=True)
        hn_top["LONGITUDE"] = hn_top.LONGITUDE + tmp_df2.LONGITUDE
        hn_top["LATITUDE"] = hn_top.LATITUDE + tmp_df2.LATITUDE
    hn_top["LONGITUDE"] = hn_top["LONGITUDE"]/(len(top_1_trips)+1)
    hn_top["LATITUDE"] = hn_top["LATITUDE"]/(len(top_1_trips)+1)
    return hn_top[["LONGITUDE", "LATITUDE"]]

In [74]:
hn_top = get_top1_location(d1)
nh_top = get_top1_location(d2)

hn_top.to_csv("H2N_top1.csv", index=False)
nh_top.to_csv("N2H_top1.csv", index=False)

In [ ]:
hn_top = pd.read_csv("H2N_top1.csv")
nh_top = pd.read_csv("N2H_top1.csv")

In [ ]:
hn_top_np = np.zeros((len(hn_top), 18))
hn_top_np[:,13] = hn_top["LATITUDE"]
hn_top_np[:,14] = hn_top["LONGITUDE"]
hn_top_np = minmax_scaler.transform(hn_top_np)
hn_top["LATITUDE"] = hn_top_np[:,13]
hn_top["LONGITUDE"] = hn_top_np[:,14]

nh_top_np = np.zeros((len(nh_top), 18))
nh_top_np[:,13] = nh_top["LATITUDE"]
nh_top_np[:,14] = nh_top["LONGITUDE"]
nh_top_np = minmax_scaler.transform(nh_top_np)
nh_top["LATITUDE"] = nh_top_np[:,13]
nh_top["LONGITUDE"] = nh_top_np[:,14]
nh_top

In [79]:
def get_observations(df, feature_cols = ["Time2", "turn", "acceleration",
       'change_x_factor', 'change_y_factor', "distance",
       'current', 'rain', 'snowfall', 'wind_force', 'wind_direc', "resist_ratio",
       "is_weekday", 'direction', "season", "hour", 
       "FC", "SOG", "LATITUDE", 'LONGITUDE',
       ], 
       action_cols = ["SPEED", "HEADING", "MODE"]):
    # df = df.fillna(0)
    
    features = df.copy()

    rewards_col = ["trip_id", "LONGITUDE", "LATITUDE", "direction", "goal_long", "goal_lat"]
    rewards_df = features[rewards_col]
    # apply minmax scaler for fuel consumption to get reward 2
    rewards_df["reward2"] = - features[["FC"]]

    dataset_list = []
    for i in list(df.trip_id.unique()):
        data_dict = {}
        # observations
        observation = features[features.trip_id==i].drop("trip_id", axis=1)[feature_cols]
        observation = observation.to_numpy().astype(float)
        data_dict["observations"] = observation

        # next_observations
        observation = np.delete(observation, 0, 0)
        last = observation[-1]
        observation = np.vstack([observation, last])
        data_dict["next_observations"] = observation

        # actions 
        # print(i)
        actions = features[features.trip_id==i].drop("trip_id", axis=1)[action_cols]
        # actions = actions.drop("trip_id", axis=1)
        data_dict["actions"] = actions.to_numpy().astype(float)

        # rewards
        rewards = rewards_df[rewards_df.trip_id==i].reset_index()
        rewards
        # reward1 distance to top 1
        trip_len = rewards.shape[0]
        if rewards.loc[0,"direction"]==1:
            top1 = hn_top.iloc[:trip_len]
        else:
            top1 = nh_top.iloc[:trip_len]
        rewards["reward1"] = - ((rewards["LONGITUDE"]-top1["LONGITUDE"])**2 + \
                             (rewards["LATITUDE"]-top1["LATITUDE"])**2 )**0.5
        rewards["reward1"].apply(lambda x: 0 if x > -0.05 else x)
        rewards["reward1"] = rewards["reward1"] * 10
        # reward2 fc consumption an ddone reward
        rewards.loc[len(rewards)-1,"reward2"] = rewards.loc[len(rewards)-1,"reward2"]+3
        # reward3 mimic reward
        rewards["reward3"] = 0
        # rewards4 time out reward
        rewards["Time"] = rewards.index
        rewards.reset_index(inplace=True)
        rewards["reward4"] = rewards["Time"].apply(lambda x: -0.1*((x-90)//10) if x > 100 else 0)

        data_dict["rewards"] = (rewards[["reward1","reward2", "reward3", "reward4"]]).to_numpy()

        # termination
        termination = np.zeros([observation.shape[0],1])
        termination[-1,0] = 1
        data_dict["termination"] = termination

        dataset_list.append(data_dict)
    return dataset_list

In [55]:
rl_data = get_observations(df)
with open('rl_data.pkl', 'wb') as handle:
    pickle.dump(rl_data,handle)

0.0